In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Media_Bias_Feature_for_the_Identification_of_Biased_Statements_in_News_Articles')
!pwd

/content
/content/drive/MyDrive/Colab Notebooks/Media_Bias_Feature_for_the_Identification_of_Biased_Statements_in_News_Articles


In [ ]:
!pip install -U scikit-learn
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.9 MB/s eta 0:00:00


In [ ]:
# import all of the packages that will be used

# basic packages for data processing
import pandas as pd
import numpy as np
import gc

# packege for calculation
import math

# packages for machine learning
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

import xgboost

# packages to interpret the training result
import torch
from torchmetrics.classification import Accuracy, ConfusionMatrix, AUROC, Precision, Recall, F1Score

# package to benchmark the CPU running time
from time import process_time

# silent the warning from sklearn package
# import warnings
# warnings.filterwarnings('ignore')

# set the random seed to ensure the result is reproducible
import random
random.seed(42)
np.random.seed(42)

In [ ]:
fdt_hb_BABESG2_train = pd.read_csv('Ground_truth_datasets_characterised/Splits/Train/BABESG2_hybrid_characterised_index_output_train.csv')
fdt_hb_BABESG2_test = pd.read_csv('Ground_truth_datasets_characterised/Splits/Test/BABESG2_hybrid_characterised_index_output_test.csv')

In [ ]:
fdt_hb_BABESG2_train.columns

Index(['text', 'bias', 'bias_BABESG2', 'objective_BABESG2',
       'partly-subjective_BABESG2', 'subjective_BABESG2',
       'nonpartisan_headline', 'slight-partisan_headline',
       'hyperpartisan_headline', 'nonpartisan_article',
       'slight-partisan_article', 'hyperpartisan_article', 'true_LIAR',
       'mostly-true_LIAR', 'half-true_LIAR', 'barely-true_LIAR', 'false_LIAR',
       'pants-fire_LIAR', 'bluff_FakeNewsTFG', 'negative_NewsMTSC',
       'neutral_NewsMTSC', 'positive_NewsMTSC'],
      dtype='object')

In [ ]:
FEATURES = ['bias_BABESG2', 'objective_BABESG2',
       'partly-subjective_BABESG2', 'subjective_BABESG2',
       'nonpartisan_headline', 'slight-partisan_headline',
       'hyperpartisan_headline', 'nonpartisan_article',
       'slight-partisan_article', 'hyperpartisan_article', 'true_LIAR',
       'mostly-true_LIAR', 'half-true_LIAR', 'barely-true_LIAR', 'false_LIAR',
       'pants-fire_LIAR', 'bluff_FakeNewsTFG', 'negative_NewsMTSC',
       'neutral_NewsMTSC', 'positive_NewsMTSC']
TARGET = ['bias']

In [ ]:
def runKFold(clf, output_type, f = 5, scaler = False):

  scaler = StandardScaler()

  val_acc = []
  val_prec = []
  val_rec = []
  val_f1 = []
  val_auroc = []

  for k in range(1, 6):
    fdt_hb_BABESG2_train_K = pd.read_csv(f'Ground_truth_datasets_characterised/Splits/Train/BABESG2_hybrid_characterised_{output_type}_output_train_KFold{k}.csv')
    fdt_hb_BABESG2_val_K = pd.read_csv(f'Ground_truth_datasets_characterised/Splits/Validation/BABESG2_hybrid_characterised_{output_type}_output_val_KFold{k}.csv')

    train_X = fdt_hb_BABESG2_train_K[FEATURES]
    val_X = fdt_hb_BABESG2_val_K[FEATURES]
    train_y = fdt_hb_BABESG2_train_K['bias']
    val_y = fdt_hb_BABESG2_val_K['bias']

    if(scaler == True):
      scaler.fit(train_X)
      train_X = scaler.transform(train_X)
      train_X = scaler.transform(val_X)

    trained_clf = clf.fit(train_X, train_y)

    preds = trained_clf.predict(val_X)
    preds = torch.from_numpy(np.array(preds))

    val_y = torch.from_numpy(val_y.to_numpy())

    accuracy = Accuracy(task = 'binary', threshold=0.5)
    precision = Precision(task = 'binary', threshold=0.5)
    recall = Recall(task = 'binary', threshold=0.5)
    f1 = F1Score(task = 'binary', threshold=0.5)
    auroc = AUROC(task = 'binary')

    val_acc.append(accuracy(preds, val_y).item())
    val_prec.append(precision(preds, val_y).item())
    val_rec.append(recall(preds, val_y).item())
    val_f1.append(f1(preds, val_y).item())
    val_auroc.append(auroc(preds, val_y).item())

    k += 1

    gc.collect()

  return val_acc, val_prec, val_rec, val_f1, val_auroc


# Media Bias Feature Effectiveness Analysis

# BABE SG2 Statement Bias + BABE SG2 Subjectivity

In [ ]:
FEATURES = ['bias_BABESG2',
            'objective_BABESG2', 'partly-subjective_BABESG2', 'subjective_BABESG2']
TARGET = 'bias'

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_GaussianNB = GaussianNB(var_smoothing = 0.004937047852839003)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_GaussianNB, 'index', 5, True)

In [ ]:
val_acc

[0.8174757361412048,
 0.7957198619842529,
 0.7898832559585571,
 0.7996109127998352,
 0.7626459002494812]

In [ ]:
val_prec

[0.8117647171020508,
 0.786821722984314,
 0.7686567306518555,
 0.7921568751335144,
 0.7462121248245239]

In [ ]:
val_rec

[0.8181818127632141,
 0.8023715615272522,
 0.817460298538208,
 0.8015872836112976,
 0.7817460298538208]

In [ ]:
val_f1

[0.8149606585502625,
 0.7945205569267273,
 0.7923076748847961,
 0.7968441843986511,
 0.7635658979415894]

In [ ]:
val_auroc

[0.8174878358840942,
 0.7958217859268188,
 0.7904095649719238,
 0.799648642539978,
 0.7630104422569275]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7930671334266662
5-Fold CV Precision: 0.7811224341392518
5-Fold CV Recall: 0.8042693972587586
5-Fold CV F1 Score: 0.7924397945404053
5-Fold CV AUROC: 0.7932756543159485


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.017783286577647703
5-Fold CV Precision: 0.022214021456968186
5-Fold CV Recall: 0.013309040349118246
5-Fold CV F1 Score: 0.016520513904141984
5-Fold CV AUROC: 0.017649504415012854


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-20-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-20-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.817476,0.811765,0.818182,0.814961,0.817488
1,2,0.795720,0.786822,0.802372,0.794521,0.795822
2,3,0.789883,0.768657,0.817460,0.792308,0.790410
3,4,0.799611,0.792157,0.801587,0.796844,0.799649
4,5,0.762646,0.746212,0.781746,0.763566,0.763010
5,cv mean,0.793067,0.781122,0.804269,0.792440,0.793276
6,cv std,0.017783,0.022214,0.013309,0.016521,0.017650


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train[TARGET]
test_y = fdt_hb_BABESG2_test[TARGET]

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_GaussianNB = GaussianNB()

In [ ]:
trained_clf = clf_GaussianNB.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[436, 118],
        [102, 446]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.8003630042076111


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.7907801270484924 
Recall: 0.8138686418533325 
F1 Score: 0.8021582961082458


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.8004361391067505


# BABE SG2 Statement Bias + News Headlines USA Hyper-partisanship

In [ ]:
FEATURES = ['bias_BABESG2',
            'nonpartisan_headline', 'slight-partisan_headline', 'hyperpartisan_headline']
TARGET = 'bias'

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_GaussianNB = GaussianNB(var_smoothing = 0.004937047852839003)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_GaussianNB, 'index', 5, True)

In [ ]:
val_acc

[0.800000011920929,
 0.7821011543273926,
 0.7762646079063416,
 0.7665369510650635,
 0.7548637986183167]

In [ ]:
val_prec

[0.8151260614395142,
 0.7722007632255554,
 0.7686274647712708,
 0.7682926654815674,
 0.7441860437393188]

In [ ]:
val_rec

[0.7667984366416931,
 0.790513813495636,
 0.7777777910232544,
 0.75,
 0.761904776096344]

In [ ]:
val_f1

[0.790224015712738,
 0.78125,
 0.7731755375862122,
 0.759036123752594,
 0.7529411911964417]

In [ ]:
val_auroc

[0.7994297742843628,
 0.7822300791740417,
 0.7762935161590576,
 0.7662214040756226,
 0.7549981474876404]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7759533047676086
5-Fold CV Precision: 0.7736865997314453
5-Fold CV Recall: 0.7693989634513855
5-Fold CV F1 Score: 0.7713253736495972
5-Fold CV AUROC: 0.775834584236145


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.015162928974600703
5-Fold CV Precision: 0.022997429742728184
5-Fold CV Recall: 0.013822351541317985
5-Fold CV F1 Score: 0.01377014026159561
5-Fold CV AUROC: 0.014996108139081575


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-45-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-45-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.800000,0.815126,0.766798,0.790224,0.799430
1,2,0.782101,0.772201,0.790514,0.781250,0.782230
2,3,0.776265,0.768627,0.777778,0.773176,0.776294
3,4,0.766537,0.768293,0.750000,0.759036,0.766221
4,5,0.754864,0.744186,0.761905,0.752941,0.754998
5,cv mean,0.775953,0.773687,0.769399,0.771325,0.775835
6,cv std,0.015163,0.022997,0.013822,0.013770,0.014996


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train[TARGET]
test_y = fdt_hb_BABESG2_test[TARGET]

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_GaussianNB = GaussianNB()

In [ ]:
trained_clf = clf_GaussianNB.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[438, 116],
        [102, 446]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.8021778464317322


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.7935943007469177 
Recall: 0.8138686418533325 
F1 Score: 0.8036035895347595


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.8022411465644836


# BABE SG2 Statement Bias + News Articles USA Hyper-partisanship

In [ ]:
FEATURES = ['bias_BABESG2',
            'nonpartisan_article', 'slight-partisan_article', 'hyperpartisan_article']
TARGET = 'bias'

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_GaussianNB = GaussianNB(var_smoothing = 0.004937047852839003)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_GaussianNB, 'index', 5, True)

In [ ]:
val_acc

[0.7883495092391968,
 0.768482506275177,
 0.7801556587219238,
 0.7859922051429749,
 0.7412451505661011]

In [ ]:
val_prec

[0.800000011920929,
 0.7481481432914734,
 0.772549033164978,
 0.7629629373550415,
 0.7297297120094299]

In [ ]:
val_rec

[0.7588932514190674,
 0.7984189987182617,
 0.7817460298538208,
 0.817460298538208,
 0.75]

In [ ]:
val_f1

[0.7789046764373779,
 0.7724665403366089,
 0.7771202921867371,
 0.7892720103263855,
 0.7397260069847107]

In [ ]:
val_auroc

[0.7878435850143433,
 0.7689412832260132,
 0.7801859974861145,
 0.7865927815437317,
 0.7414122819900513]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7728450059890747
5-Fold CV Precision: 0.7626779675483704
5-Fold CV Recall: 0.7813037157058715
5-Fold CV F1 Score: 0.771497905254364
5-Fold CV AUROC: 0.7729951858520507


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.017229430498494524
5-Fold CV Precision: 0.023609987193308476
5-Fold CV Recall: 0.024829734445037666
5-Fold CV F1 Score: 0.016809414498226042
5-Fold CV AUROC: 0.017152208493902303


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-70-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-70-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.788350,0.800000,0.758893,0.778905,0.787844
1,2,0.768483,0.748148,0.798419,0.772467,0.768941
2,3,0.780156,0.772549,0.781746,0.777120,0.780186
3,4,0.785992,0.762963,0.817460,0.789272,0.786593
4,5,0.741245,0.729730,0.750000,0.739726,0.741412
5,cv mean,0.772845,0.762678,0.781304,0.771498,0.772995
6,cv std,0.017229,0.023610,0.024830,0.016809,0.017152


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train[TARGET]
test_y = fdt_hb_BABESG2_test[TARGET]

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_GaussianNB = GaussianNB()

In [ ]:
trained_clf = clf_GaussianNB.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[437, 117],
        [ 99, 449]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.803992748260498


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.7932862043380737 
Recall: 0.819343090057373 
F1 Score: 0.8061041235923767


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.8040758371353149


# BABE SG2 Statement Bias + LIAR Fake News

In [ ]:
FEATURES = ['bias_BABESG2',
            'true_LIAR', 'mostly-true_LIAR', 'half-true_LIAR', 'barely-true_LIAR', 'false_LIAR', 'pants-fire_LIAR']
TARGET = 'bias'

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_GaussianNB = GaussianNB(var_smoothing = 0.004937047852839003)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_GaussianNB, 'index', 5, True)

In [ ]:
val_acc

[0.800000011920929,
 0.774319052696228,
 0.7665369510650635,
 0.7879377603530884,
 0.7568093538284302]

In [ ]:
val_prec

[0.8232758641242981,
 0.7624521255493164,
 0.7639999985694885,
 0.7991631627082825,
 0.7470816969871521]

In [ ]:
val_rec

[0.7549406886100769,
 0.7865612506866455,
 0.7579365372657776,
 0.7579365372657776,
 0.761904776096344]

In [ ]:
val_f1

[0.7876288890838623,
 0.774319052696228,
 0.7609561681747437,
 0.7780040502548218,
 0.7544204592704773]

In [ ]:
val_auroc

[0.7992261052131653,
 0.774506688117981,
 0.7663728594779968,
 0.787365198135376,
 0.7569065690040588]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7771206259727478
5-Fold CV Precision: 0.7791945695877075
5-Fold CV Recall: 0.7638559579849243
5-Fold CV F1 Score: 0.7710657238960266
5-Fold CV AUROC: 0.7768754839897156


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.015315922832555439
5-Fold CV Precision: 0.027872302548238047
5-Fold CV Recall: 0.01156631781302742
5-Fold CV F1 Score: 0.011935919849925203
5-Fold CV AUROC: 0.014995054205652456


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-95-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-95-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.800000,0.823276,0.754941,0.787629,0.799226
1,2,0.774319,0.762452,0.786561,0.774319,0.774507
2,3,0.766537,0.764000,0.757937,0.760956,0.766373
3,4,0.787938,0.799163,0.757937,0.778004,0.787365
4,5,0.756809,0.747082,0.761905,0.754420,0.756907
5,cv mean,0.777121,0.779195,0.763856,0.771066,0.776875
6,cv std,0.015316,0.027872,0.011566,0.011936,0.014995


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train[TARGET]
test_y = fdt_hb_BABESG2_test[TARGET]

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_GaussianNB = GaussianNB()

In [ ]:
trained_clf = clf_GaussianNB.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[445, 109],
        [103, 445]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.807622492313385


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.8032491207122803 
Recall: 0.8120437860488892 
F1 Score: 0.807622492313385


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.8076464533805847


# BABE SG2 Statement Bias + Fake News TFG Fake News

In [ ]:
FEATURES = ['bias_BABESG2',
            'bluff_FakeNewsTFG']
TARGET = 'bias'

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_GaussianNB = GaussianNB(var_smoothing = 0.004937047852839003)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_GaussianNB, 'index', 5, True)

In [ ]:
val_acc

[0.7961165308952332,
 0.7840467095375061,
 0.7704280018806458,
 0.7723735570907593,
 0.7587548494338989]

In [ ]:
val_prec

[0.8135592937469482,
 0.7709923386573792,
 0.7658730149269104,
 0.7777777910232544,
 0.748062014579773]

In [ ]:
val_rec

[0.7588932514190674,
 0.7984189987182617,
 0.7658730149269104,
 0.75,
 0.7658730149269104]

In [ ]:
val_f1

[0.7852760553359985,
 0.784466028213501,
 0.7658730149269104,
 0.7636363506317139,
 0.7568627595901489]

In [ ]:
val_auroc

[0.7954771518707275,
 0.784267008304596,
 0.7703410983085632,
 0.7719465494155884,
 0.7588907480239868]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7763439297676087
5-Fold CV Precision: 0.775252890586853
5-Fold CV Recall: 0.76781165599823
5-Fold CV F1 Score: 0.7712228417396545
5-Fold CV AUROC: 0.7761845111846923


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.012731271582039171
5-Fold CV Precision: 0.021537101724464335
5-Fold CV Recall: 0.01637754020862232
5-Fold CV F1 Score: 0.011534836400775744
5-Fold CV AUROC: 0.012559560363241896


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-120-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-120-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.796117,0.813559,0.758893,0.785276,0.795477
1,2,0.784047,0.770992,0.798419,0.784466,0.784267
2,3,0.770428,0.765873,0.765873,0.765873,0.770341
3,4,0.772374,0.777778,0.750000,0.763636,0.771947
4,5,0.758755,0.748062,0.765873,0.756863,0.758891
5,cv mean,0.776344,0.775253,0.767812,0.771223,0.776185
6,cv std,0.012731,0.021537,0.016378,0.011535,0.012560


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train[TARGET]
test_y = fdt_hb_BABESG2_test[TARGET]

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_GaussianNB = GaussianNB()

In [ ]:
trained_clf = clf_GaussianNB.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[438, 116],
        [101, 447]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.8030853271484375


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.7939609289169312 
Recall: 0.8156934380531311 
F1 Score: 0.8046804666519165


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.8031535148620605


# BABE SG2 Statement Bias + NewsMTSC News Sentiment

In [ ]:
FEATURES = ['bias_BABESG2',
            'negative_NewsMTSC', 'neutral_NewsMTSC', 'positive_NewsMTSC']
TARGET = 'bias'

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_GaussianNB = GaussianNB(var_smoothing = 0.004937047852839003)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_GaussianNB, 'index', 5, True)

In [ ]:
val_acc

[0.7941747307777405,
 0.7587548494338989,
 0.7723735570907593,
 0.7665369510650635,
 0.7548637986183167]

In [ ]:
val_prec

[0.8127659559249878,
 0.7397769689559937,
 0.7710843086242676,
 0.7661290168762207,
 0.7441860437393188]

In [ ]:
val_rec

[0.7549406886100769,
 0.7865612506866455,
 0.761904776096344,
 0.7539682388305664,
 0.761904776096344]

In [ ]:
val_f1

[0.7827869057655334,
 0.7624521255493164,
 0.7664670944213867,
 0.7599999904632568,
 0.7529411911964417]

In [ ]:
val_auroc

[0.7935009002685547,
 0.759181022644043,
 0.7721737027168274,
 0.7662971019744873,
 0.7549981474876404]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7693407773971558
5-Fold CV Precision: 0.7667884588241577
5-Fold CV Recall: 0.7638559460639953
5-Fold CV F1 Score: 0.7649294614791871
5-Fold CV AUROC: 0.7692301750183106


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.013823380344081863
5-Fold CV Precision: 0.025974253334446102
5-Fold CV Recall: 0.011835485256602867
5-Fold CV F1 Score: 0.009953975423752901
5-Fold CV AUROC: 0.013489784639053199


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-145-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-145-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.794175,0.812766,0.754941,0.782787,0.793501
1,2,0.758755,0.739777,0.786561,0.762452,0.759181
2,3,0.772374,0.771084,0.761905,0.766467,0.772174
3,4,0.766537,0.766129,0.753968,0.760000,0.766297
4,5,0.754864,0.744186,0.761905,0.752941,0.754998
5,cv mean,0.769341,0.766788,0.763856,0.764929,0.769230
6,cv std,0.013823,0.025974,0.011835,0.009954,0.013490


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train[TARGET]
test_y = fdt_hb_BABESG2_test[TARGET]

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_GaussianNB = GaussianNB()

In [ ]:
trained_clf = clf_GaussianNB.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[435, 119],
        [102, 446]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.7994555234909058


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.7893805503845215 
Recall: 0.8138686418533325 
F1 Score: 0.8014375567436218


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.7995336055755615


# Media bias feature combinations analysis

# BABE SG2 Statement Bias + BABE SG2 Subjectivity

In [ ]:
FEATURES = ['bias_BABESG2', 'objective_BABESG2', 'partly-subjective_BABESG2', 'subjective_BABESG2']
TARGET = 'bias'

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_GaussianNB = GaussianNB(var_smoothing = 0.004937047852839003)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_GaussianNB, 'index', 5, True)

In [ ]:
val_acc

[0.8174757361412048,
 0.7957198619842529,
 0.7898832559585571,
 0.7996109127998352,
 0.7626459002494812]

In [ ]:
val_prec

[0.8117647171020508,
 0.786821722984314,
 0.7686567306518555,
 0.7921568751335144,
 0.7462121248245239]

In [ ]:
val_rec

[0.8181818127632141,
 0.8023715615272522,
 0.817460298538208,
 0.8015872836112976,
 0.7817460298538208]

In [ ]:
val_f1

[0.8149606585502625,
 0.7945205569267273,
 0.7923076748847961,
 0.7968441843986511,
 0.7635658979415894]

In [ ]:
val_auroc

[0.8174878358840942,
 0.7958217859268188,
 0.7904095649719238,
 0.799648642539978,
 0.7630104422569275]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7930671334266662
5-Fold CV Precision: 0.7811224341392518
5-Fold CV Recall: 0.8042693972587586
5-Fold CV F1 Score: 0.7924397945404053
5-Fold CV AUROC: 0.7932756543159485


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.017783286577647703
5-Fold CV Precision: 0.022214021456968186
5-Fold CV Recall: 0.013309040349118246
5-Fold CV F1 Score: 0.016520513904141984
5-Fold CV AUROC: 0.017649504415012854


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-20-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-20-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.817476,0.811765,0.818182,0.814961,0.817488
1,2,0.795720,0.786822,0.802372,0.794521,0.795822
2,3,0.789883,0.768657,0.817460,0.792308,0.790410
3,4,0.799611,0.792157,0.801587,0.796844,0.799649
4,5,0.762646,0.746212,0.781746,0.763566,0.763010
5,cv mean,0.793067,0.781122,0.804269,0.792440,0.793276
6,cv std,0.017783,0.022214,0.013309,0.016521,0.017650


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train[TARGET]
test_y = fdt_hb_BABESG2_test[TARGET]

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_GaussianNB = GaussianNB()

In [ ]:
trained_clf = clf_GaussianNB.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[436, 118],
        [102, 446]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.8003630042076111


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.7907801270484924 
Recall: 0.8138686418533325 
F1 Score: 0.8021582961082458


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.8004361391067505


# BABE SG2 Statement Bias + BABE SG2 Subjectivity + News Headlines USA Hyper-partisanship

In [ ]:
FEATURES = ['bias_BABESG2', 'objective_BABESG2', 'partly-subjective_BABESG2', 'subjective_BABESG2',
            'nonpartisan_headline', 'slight-partisan_headline', 'hyperpartisan_headline']
TARGET = 'bias'

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_GaussianNB = GaussianNB(var_smoothing = 0.004937047852839003)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_GaussianNB, 'index', 5, True)

In [ ]:
val_acc

[0.8194174766540527,
 0.7937743067741394,
 0.7898832559585571,
 0.7996109127998352,
 0.774319052696228]

In [ ]:
val_prec

[0.8100775480270386,
 0.790513813495636,
 0.7769230604171753,
 0.7968127727508545,
 0.7615384459495544]

In [ ]:
val_rec

[0.8260869383811951,
 0.790513813495636,
 0.8015872836112976,
 0.7936508059501648,
 0.7857142686843872]

In [ ]:
val_f1

[0.8180038928985596,
 0.790513813495636,
 0.7890625,
 0.7952286005020142,
 0.7734375]

In [ ]:
val_auroc

[0.819532036781311,
 0.7937242984771729,
 0.7901066541671753,
 0.7994971871376038,
 0.7745365500450134]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7954010009765625
5-Fold CV Precision: 0.7871731281280517
5-Fold CV Recall: 0.7995106220245362
5-Fold CV F1 Score: 0.7932492613792419
5-Fold CV AUROC: 0.7954793453216553


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.014642777393198525
5-Fold CV Precision: 0.016678706164198375
5-Fold CV Recall: 0.01425645459469175
5-Fold CV F1 Score: 0.014382121539875744
5-Fold CV AUROC: 0.014596038455543124


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-170-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-170-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.819417,0.810078,0.826087,0.818004,0.819532
1,2,0.793774,0.790514,0.790514,0.790514,0.793724
2,3,0.789883,0.776923,0.801587,0.789062,0.790107
3,4,0.799611,0.796813,0.793651,0.795229,0.799497
4,5,0.774319,0.761538,0.785714,0.773438,0.774537
5,cv mean,0.795401,0.787173,0.799511,0.793249,0.795479
6,cv std,0.014643,0.016679,0.014256,0.014382,0.014596


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train[TARGET]
test_y = fdt_hb_BABESG2_test[TARGET]

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_GaussianNB = GaussianNB()

In [ ]:
trained_clf = clf_GaussianNB.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[438, 116],
        [ 97, 451]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.8067150712013245


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.7954144477844238 
Recall: 0.8229926824569702 
F1 Score: 0.8089686036109924


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.8068031668663025


# BABE SG2 Statement Bias + BABE SG2 Subjectivity + News Headlines USA Hyper-partisanship + News Articles USA Hyper-partisanship

In [ ]:
FEATURES = ['bias_BABESG2',
            'objective_BABESG2', 'partly-subjective_BABESG2', 'subjective_BABESG2',
            'nonpartisan_headline', 'slight-partisan_headline', 'hyperpartisan_headline',
            'nonpartisan_article', 'slight-partisan_article', 'hyperpartisan_article']
TARGET = 'bias'

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_GaussianNB = GaussianNB(var_smoothing = 0.004937047852839003)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_GaussianNB, 'index', 5, True)

In [ ]:
val_acc

[0.8233009576797485,
 0.801556408405304,
 0.7918288111686707,
 0.7898832559585571,
 0.7801556587219238]

In [ ]:
val_prec

[0.8139534592628479,
 0.7915058135986328,
 0.7735849022865295,
 0.7790697813034058,
 0.7683397531509399]

In [ ]:
val_rec

[0.8300395011901855,
 0.8102766871452332,
 0.8134920597076416,
 0.7976190447807312,
 0.7896825671195984]

In [ ]:
val_f1

[0.8219178318977356,
 0.80078125,
 0.7930367588996887,
 0.7882353067398071,
 0.7788649797439575]

In [ ]:
val_auroc

[0.8234167098999023,
 0.8016901016235352,
 0.7922422289848328,
 0.7900309562683105,
 0.7803375124931335]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7973450183868408
5-Fold CV Precision: 0.7852907419204712
5-Fold CV Recall: 0.808221971988678
5-Fold CV F1 Score: 0.7965672254562378
5-Fold CV AUROC: 0.7975435018539428


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.014649417167686472
5-Fold CV Precision: 0.016268014840250682
5-Fold CV Recall: 0.01388619355817464
5-Fold CV F1 Score: 0.014530421994348303
5-Fold CV AUROC: 0.014609620729724198


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-195-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-195-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.823301,0.813953,0.830040,0.821918,0.823417
1,2,0.801556,0.791506,0.810277,0.800781,0.801690
2,3,0.791829,0.773585,0.813492,0.793037,0.792242
3,4,0.789883,0.779070,0.797619,0.788235,0.790031
4,5,0.780156,0.768340,0.789683,0.778865,0.780338
5,cv mean,0.797345,0.785291,0.808222,0.796567,0.797544
6,cv std,0.014649,0.016268,0.013886,0.014530,0.014610


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train[TARGET]
test_y = fdt_hb_BABESG2_test[TARGET]

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_GaussianNB = GaussianNB()

In [ ]:
trained_clf = clf_GaussianNB.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[443, 111],
        [ 96, 452]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.8121597170829773


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.8028419017791748 
Recall: 0.8248175382614136 
F1 Score: 0.8136813640594482


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.812228262424469


# BABE SG2 Statement Bias + BABE SG2 Subjectivity + News Headlines USA Hyper-partisanship + News Articles USA Hyper-partisanship + LIAR Fake News

In [ ]:
FEATURES = ['bias_BABESG2',
            'objective_BABESG2', 'partly-subjective_BABESG2', 'subjective_BABESG2',
            'nonpartisan_headline', 'slight-partisan_headline', 'hyperpartisan_headline',
            'nonpartisan_article', 'slight-partisan_article', 'hyperpartisan_article',
            'true_LIAR', 'mostly-true_LIAR', 'half-true_LIAR', 'barely-true_LIAR', 'false_LIAR', 'pants-fire_LIAR']
TARGET = 'bias'

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_GaussianNB = GaussianNB(var_smoothing = 0.004937047852839003)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_GaussianNB, 'index', 5, True)

In [ ]:
val_acc

[0.8233009576797485,
 0.8054474592208862,
 0.7782101035118103,
 0.7879377603530884,
 0.774319052696228]

In [ ]:
val_prec

[0.81640625,
 0.7976653575897217,
 0.7593985199928284,
 0.7718631029129028,
 0.7615384459495544]

In [ ]:
val_rec

[0.8260869383811951,
 0.8102766871452332,
 0.8015872836112976,
 0.8055555820465088,
 0.7857142686843872]

In [ ]:
val_f1

[0.8212180733680725,
 0.8039215803146362,
 0.7799227833747864,
 0.7883495092391968,
 0.7734375]

In [ ]:
val_auroc

[0.823348879814148,
 0.8055214881896973,
 0.7786562442779541,
 0.7882739901542664,
 0.7745365500450134]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7938430666923523
5-Fold CV Precision: 0.7813743352890015
5-Fold CV Recall: 0.8058441519737244
5-Fold CV F1 Score: 0.7933698892593384
5-Fold CV AUROC: 0.7940674304962159


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.018236658524341587
5-Fold CV Precision: 0.022178796004525082
5-Fold CV Recall: 0.013060123825488039
5-Fold CV F1 Score: 0.017263768743140548
5-Fold CV AUROC: 0.0181169678183357


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-46-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-46-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.823301,0.816406,0.826087,0.821218,0.823349
1,2,0.805447,0.797665,0.810277,0.803922,0.805521
2,3,0.778210,0.759399,0.801587,0.779923,0.778656
3,4,0.787938,0.771863,0.805556,0.788350,0.788274
4,5,0.774319,0.761538,0.785714,0.773438,0.774537
5,cv mean,0.793843,0.781374,0.805844,0.793370,0.794067
6,cv std,0.018237,0.022179,0.013060,0.017264,0.018117


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train[TARGET]
test_y = fdt_hb_BABESG2_test[TARGET]

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_GaussianNB = GaussianNB()

In [ ]:
trained_clf = clf_GaussianNB.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[447, 107],
        [ 98, 450]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.8139746189117432


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.8078994750976562 
Recall: 0.8211678862571716 
F1 Score: 0.814479649066925


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.8140136003494263


# BABE SG2 Statement Bias + BABE SG2 Subjectivity + News Headlines USA Hyper-partisanship + News Articles USA Hyper-partisanship + LIAR Fake News + Fake News TFG Fake News

In [ ]:
FEATURES = ['bias_BABESG2',
            'objective_BABESG2', 'partly-subjective_BABESG2', 'subjective_BABESG2',
            'nonpartisan_headline', 'slight-partisan_headline', 'hyperpartisan_headline',
            'nonpartisan_article', 'slight-partisan_article', 'hyperpartisan_article',
            'true_LIAR', 'mostly-true_LIAR', 'half-true_LIAR', 'barely-true_LIAR', 'false_LIAR', 'pants-fire_LIAR',
            'bluff_FakeNewsTFG']
TARGET = 'bias'

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_GaussianNB = GaussianNB(var_smoothing = 0.004937047852839003)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_GaussianNB, 'index', 5, True)

In [ ]:
val_acc

[0.8252426981925964,
 0.8035019636154175,
 0.7782101035118103,
 0.7840467095375061,
 0.7762646079063416]

In [ ]:
val_prec

[0.8221343755722046,
 0.796875,
 0.7613636255264282,
 0.7722007632255554,
 0.7644787430763245]

In [ ]:
val_rec

[0.8221343755722046,
 0.8063241243362427,
 0.7976190447807312,
 0.7936508059501648,
 0.7857142686843872]

In [ ]:
val_f1

[0.8221343755722046,
 0.8015717267990112,
 0.7790697813034058,
 0.7827788591384888,
 0.7749511003494263]

In [ ]:
val_auroc

[0.8251892924308777,
 0.8035451769828796,
 0.7785805463790894,
 0.7842299938201904,
 0.7764449119567871]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7934532165527344
5-Fold CV Precision: 0.7834105014801025
5-Fold CV Recall: 0.801088523864746
5-Fold CV F1 Score: 0.7921011686325073
5-Fold CV AUROC: 0.7935979843139649


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.018589414176168585
5-Fold CV Precision: 0.023025922749824707
5-Fold CV Recall: 0.012442379528302143
5-Fold CV F1 Score: 0.017564352212395957
5-Fold CV AUROC: 0.018463303982982546


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-45-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-45-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.825243,0.822134,0.822134,0.822134,0.825189
1,2,0.803502,0.796875,0.806324,0.801572,0.803545
2,3,0.778210,0.761364,0.797619,0.779070,0.778581
3,4,0.784047,0.772201,0.793651,0.782779,0.784230
4,5,0.776265,0.764479,0.785714,0.774951,0.776445
5,cv mean,0.793453,0.783411,0.801089,0.792101,0.793598
6,cv std,0.018589,0.023026,0.012442,0.017564,0.018463


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train[TARGET]
test_y = fdt_hb_BABESG2_test[TARGET]

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_GaussianNB = GaussianNB()

In [ ]:
trained_clf = clf_GaussianNB.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[449, 105],
        [ 97, 451]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.8166969418525696


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.8111510872840881 
Recall: 0.8229926824569702 
F1 Score: 0.8170289993286133


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.8167310357093811


# BABE SG2 Statement Bias + BABE SG2 Subjectivity + News Headlines USA Hyper-partisanship + News Articles USA Hyper-partisanship + LIAR Fake News + Fake News TFG Fake News + NewsMTSC News Sentiment

In [ ]:
FEATURES = ['bias_BABESG2',
            'objective_BABESG2', 'partly-subjective_BABESG2', 'subjective_BABESG2',
            'nonpartisan_headline', 'slight-partisan_headline', 'hyperpartisan_headline',
            'nonpartisan_article', 'slight-partisan_article', 'hyperpartisan_article',
            'true_LIAR', 'mostly-true_LIAR', 'half-true_LIAR', 'barely-true_LIAR', 'false_LIAR', 'pants-fire_LIAR',
            'bluff_FakeNewsTFG',
            'negative_NewsMTSC', 'neutral_NewsMTSC', 'positive_NewsMTSC']
TARGET = 'bias'

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_GaussianNB = GaussianNB(var_smoothing = 0.004937047852839003)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_GaussianNB, 'index', 5, True)

In [ ]:
val_acc

[0.8174757361412048,
 0.801556408405304,
 0.7801556587219238,
 0.7918288111686707,
 0.7704280018806458]

In [ ]:
val_prec

[0.802281379699707,
 0.7937743067741394,
 0.7642585635185242,
 0.7799227833747864,
 0.7537878751754761]

In [ ]:
val_rec

[0.8339921236038208,
 0.8063241243362427,
 0.7976190447807312,
 0.8015872836112976,
 0.7896825671195984]

In [ ]:
val_f1

[0.817829430103302,
 0.800000011920929,
 0.7805825471878052,
 0.790606677532196,
 0.7713178396224976]

In [ ]:
val_auroc

[0.8177594542503357,
 0.8016294836997986,
 0.780488908290863,
 0.7920150756835938,
 0.7707955837249756]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7922889232635498
5-Fold CV Precision: 0.7788049817085266
5-Fold CV Recall: 0.8058410286903381
5-Fold CV F1 Score: 0.7920673012733459
5-Fold CV AUROC: 0.7925377011299133


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.016404813434313038
5-Fold CV Precision: 0.017966220178823426
5-Fold CV Recall: 0.01509641529086539
5-Fold CV F1 Score: 0.01606977496412513
5-Fold CV AUROC: 0.016352146992069216


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-96-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-96-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.817476,0.802281,0.833992,0.817829,0.817759
1,2,0.801556,0.793774,0.806324,0.800000,0.801629
2,3,0.780156,0.764259,0.797619,0.780583,0.780489
3,4,0.791829,0.779923,0.801587,0.790607,0.792015
4,5,0.770428,0.753788,0.789683,0.771318,0.770796
5,cv mean,0.792289,0.778805,0.805841,0.792067,0.792538
6,cv std,0.016405,0.017966,0.015096,0.016070,0.016352


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train[TARGET]
test_y = fdt_hb_BABESG2_test[TARGET]

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_GaussianNB = GaussianNB()

In [ ]:
trained_clf = clf_GaussianNB.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[449, 105],
        [101, 447]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.8130671381950378


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.8097826242446899 
Recall: 0.8156934380531311 
F1 Score: 0.8127272725105286


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.8130813837051392
